<a href="https://colab.research.google.com/github/yarkalm/inter_group_test/blob/main/InterGroup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [132]:
import numpy as np
import pandas as pd
from random import seed
from keras import models
from keras import Sequential
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from keras.utils import to_categorical
from tensorflow.random import set_seed
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import StandardScaler, Normalizer

# Загрузка данных и предобработка

In [133]:
sd = 2
seed(sd)
np.random.seed(sd)
set_seed(sd)

In [134]:
df = pd.read_csv('/content/dataset.csv')

In [135]:
df = df.sort_values(by=['symbol','time'])

In [136]:
df.head(100)

,symbol,price,time
59,AAVEUSDT,81.4,2023-04-15 07:04:42.914
192,AAVEUSDT,81.3,2023-04-15 07:06:48.448
325,AAVEUSDT,81.5,2023-04-15 07:09:01.928
458,AAVEUSDT,81.4,2023-04-15 07:11:03.419
591,AAVEUSDT,81.4,2023-04-15 07:13:02.779
...,...,...,...
12666,AAVEUSDT,81.3,2023-04-16 20:23:03.002000
12799,AAVEUSDT,81.4,2023-04-16 20:24:51.114000
12932,AAVEUSDT,81.5,2023-04-16 20:26:45.795000
13065,AAVEUSDT,81.5,2023-04-16 20:28:36.066000


Разделение даты

In [137]:
year = []
month = []
day = []
hour = []
minute = []
second = []

for i in range(len(df)):
  date = df.iloc[i][2].split(' ')[0]
  time = df.iloc[i][2].split(' ')[1]
  year.append(int(date.split('-')[0]))
  month.append(int(date.split('-')[1]))
  day.append(int(date.split('-')[2]))
  hour.append(int(time[:2]))
  minute.append(int(time[3:5]))
  second.append(int(time[6:8]))

In [138]:
df['year'] = year
df['month'] = month 
df['day'] = day 
df['hour'] = hour
df['minute'] = minute
df['second'] = second

In [139]:
df = df.sort_values(by=['symbol','time'])

Кодирование валют в числовой вид

In [140]:
uniq_cur = list(df['symbol'].unique())
dict_cur = {uniq_cur[i]: i for i in range(len(uniq_cur))}

In [141]:
print(dict_cur)

{'AAVEUSDT': 0, 'AGIXUSDT': 1, 'AGLDUSDT': 2, 'AIONUSDT': 3, 'AKROUSDT': 4, 'ALCXUSDT': 5, 'ALGOUSDT': 6, 'ANKRUSDT': 7, 'API3USDT': 8, 'ARDRUSDT': 9, 'ARPAUSDT': 10, 'ASTRUSDT': 11, 'ATOMUSDT': 12, 'AUTOUSDT': 13, 'AVAXUSDT': 14, 'BAKEUSDT': 15, 'BANDUSDT': 16, 'BEAMUSDT': 17, 'BEARUSDT': 18, 'BETAUSDT': 19, 'BETHUSDT': 20, 'BICOUSDT': 21, 'BIFIUSDT': 22, 'BKRWUSDT': 23, 'BONDUSDT': 24, 'BTTCUSDT': 25, 'BULLUSDT': 26, 'BUSDUSDT': 27, 'BZRXUSDT': 28, 'CAKEUSDT': 29, 'CELOUSDT': 30, 'CELRUSDT': 31, 'CITYUSDT': 32, 'COMPUSDT': 33, 'COTIUSDT': 34, 'CTSIUSDT': 35, 'CTXCUSDT': 36, 'DASHUSDT': 37, 'DATAUSDT': 38, 'DEGOUSDT': 39, 'DENTUSDT': 40, 'DEXEUSDT': 41, 'DOCKUSDT': 42, 'DODOUSDT': 43, 'DOGEUSDT': 44, 'DREPUSDT': 45, 'DUSKUSDT': 46, 'DYDXUSDT': 47, 'EGLDUSDT': 48, 'FARMUSDT': 49, 'FIDAUSDT': 50, 'FIROUSDT': 51, 'FLOWUSDT': 52, 'FLUXUSDT': 53, 'GALAUSDT': 54, 'GHSTUSDT': 55, 'GLMRUSDT': 56, 'HARDUSDT': 57, 'HBARUSDT': 58, 'HIFIUSDT': 59, 'HIGHUSDT': 60, 'HIVEUSDT': 61, 'HOOKUSDT': 62, '

In [142]:
df['symbol'] = df['symbol'].map(dict_cur)

In [143]:
df['symbol'] = np.asarray(df['symbol']).astype(np.int32)

In [144]:
cat_symb = (to_categorical(df[['symbol']].to_numpy(),len(dict_cur)))

In [145]:
split = len(df)*85//100
split

12750

In [146]:
data, target = df[['symbol','year','month','day','hour','minute','second']], df[['price']].to_numpy()

In [147]:
target

array([[  81.4],
       [  81.3],
       [  81.5],
       ...,
       [1131.3],
       [1130.9],
       [1131.2]])

Стандартизация данных

In [148]:
scaler = StandardScaler()
data = scaler.fit_transform(data)
#target = scaler.fit_transform(target)

In [150]:
data = pd.DataFrame(data)


In [154]:
data[0] = cat_symb

In [155]:
data[0]

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
14996    0.0
14997    0.0
14998    0.0
14999    0.0
15000    0.0
Name: 0, Length: 15001, dtype: float32

Создание датасета

In [51]:
x = []
y = []
timesteps = 100
for i in range(timesteps, len(data)):
# Кодирование даты
  x.append(np.array(data[i-timesteps:i]))
  y.append(target[i])
x, y = np.array(x), np.array(y)

ValueError: ignored

In [ ]:
x_train, x_test = x[:split], x[split:]
y_train, y_test = y[:split], y[split:]

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

# Модель

In [ ]:
model = Sequential()
model.add(LSTM(12, input_shape=(x_train.shape[1], x_train.shape[-1]), return_sequences=True))
model.add(Dropout(0.15))
model.add(LSTM(units = 2))
model.add(Dropout(0.15))
model.add(Dense(units = 1))

In [ ]:
model.compile(loss = 'mae', optimizer = Adam(learning_rate=0.11),metrics = ['mse'])

In [ ]:
filepath="/content/weights--{loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
history = model.fit(x_train, y_train, epochs=20, batch_size=32,validation_split=0.1, verbose=2,callbacks = callbacks_list)

# Оценка решения

In [ ]:
fig = plt.figure(figsize=(18, 4))

ax = fig.add_subplot(121)
ax.plot(history.history['mse'],'--',color='black')
ax.plot(history.history['val_mse'],color='black')
ax.set_title('График точности')
ax.set_ylabel('accuracy')
ax.set_xlabel('epoch')
ax.legend(['train', 'test'], loc='upper left')
ax.grid(True)
ax2 = fig.add_subplot(122)
ax2.plot(history.history['loss'],'--',color='black')
ax2.plot(history.history['val_loss'],color='black')
ax2.set_title('График функции потерь')
ax2.set_ylabel('loss')
ax2.set_xlabel('epoch')
ax2.legend(['train', 'test'], loc='upper left')
ax2.grid(True)
plt.show()


In [ ]:
test = pd.DataFrame()
test['symbol'] = df['symbol'][split+timesteps:]
test['year'] = year[split+timesteps:]
test['month'] = month[split+timesteps:]
test['day'] = day[split+timesteps:]
test['hour'] = hour[split+timesteps:]
test['minute'] = minute[split+timesteps:]
test['second'] = second[split+timesteps:]
y_pred = list(model.predict(x_test))
test['y_pred'] = scaler.inverse_transform(y_pred)
test['y_real'] = scaler.inverse_transform(y_test)
test['отклонение'] = (test['y_pred'] - test['y_real'])
test['%'] = abs(test['отклонение']/test['y_real'])*100
test['y_pred'] = test['y_pred']

test.describe()

In [ ]:
test.head(20)